# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
import os

home = os.path.expanduser("~")

db_name=  'emb_enfsi_2011' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/andrea/anaconda3/envs/sql-face/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[12:37:19] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[12:37:19] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/andrea/anaconda3/envs/sql-face/lib/python3.8/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[12:37:19] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8100 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
[12:37:28] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgra

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [3]:
from sql_face.sqldb import SQLDataBase

2023-01-21 12:37:28.446195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-21 12:37:29.663834: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/andrea/anaconda3/envs/sql-face/lib/python3.8/site-packages/cv2/../../lib64:/home/andrea/Documents/cuda_software/cuda/lib64:/usr/local/cuda-11.2/lib64
2023-01-21 12:37:29.663922: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or di

In [4]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [5]:
database.__dict__

{'db_name': 'emb_enfsi_2011',
 'input_dir': '/home/andrea/video_resources/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/video_resources/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f5274185b20>,
 'databases': [<sql_face.databases.Enfsi at 0x7f5274185b80>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['QMagFace'],
 'quality_model_names': ['ser_fiq', 'tface']}

In [6]:
# import pandas as pd
# from tqdm import tqdm
# from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage

# session = database.session

# """ pd.read_sql_query(
#     sql = session.query(Students.first_name,
#                         Students.last_name).filter(
#       Students.score > 80).statement, con = engine) """

# # generamos un datafame con la información que necesitamos para los registros sin quality
# # hacemos una query para actualizar los registros que no tienen quality


# df_a = pd.read_sql_query(session.query(QualityImage,FaceImage.embeddingModel_id,FaceImage.croppedImage_id)
#     .join(FaceImage)
#     .filter(QualityImage.quality != None,FaceImage.embeddingModel_id == 1).statement, con=session.bind)


# sql_b = (session.query(QualityImage, FaceImage)
#     .join(FaceImage)
#     .filter(QualityImage.quality == None).all())
# n1 = 100
# n = n1
# for Q,F in tqdm(sql_b):
#     #recorremos el resultado y actualizamos el objeto QualityImage


#     qm = Q.qualityModel_id
#     em = F.embeddingModel_id
#     ci = F.croppedImage_id

#     df_r = df_a.loc[(df_a['qualityModel_id'] == qm) & (df_a['croppedImage_id'] == ci) & (df_a['embeddingModel_id']== 1)]
#     assert len(df_r) == 1
#     Q.quality = float(df_r.quality)
#     if not n:
#         session.commit()
#         n = n1
#     else:
#         n-=1
# session.commit()



In [7]:
database.create_tables(serfiq=serfiq)

Creating CroppedImages for detector mtcnn: 0it [00:00, ?it/s]
Face images in QMagFace: 0it [00:00, ?it/s]00<?, ?it/s]
Quality images in ser_fiq: 100%|██████████| 75/75 [00:00<00:00, 62070.40it/s]
Quality images in tface: 0it [00:00, ?it/s]
Quality models: 100%|██████████| 2/2 [00:00<00:00, 216.14it/s]


In [8]:
database.update_tables(attributes_to_update, serfiq = serfiq)

Update cropped images: 0it [00:00, ?it/s]
Computing embeddings DeepFace: 0it [00:00, ?it/s]


=> loading pth from models/qmagface/magface_epoch_00025.pth ...


Computing embeddings QMagFace:   0%|          | 0/375 [00:10<?, ?it/s]


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[1152,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]